In [267]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

ss = StandardScaler()

In [319]:
df = pd.read_excel('/Users/dear/Desktop/imbalance&return20.xlsx',sheet_name= 'huobidm')

In [362]:
name = ['average imbalance', 'order imbalance ratio', 'net trade size', 'depth imbalance']
data = df[name]
look_back = 20
for i in range(1, look_back+1, 1):
    data['average imbalance - ' + str(i) + 's'] = data['average imbalance'].shift(i)
    data['net trade size - ' + str(i) + 's'] = data['net trade size'].shift(i)
    data['order imbalance ratio - ' + str(i) + 's'] = data['order imbalance ratio'].shift(i)
    data['depth imbalance - ' + str(i) + 's'] = data['depth imbalance'].shift(i)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [363]:
# data.columns

In [364]:
data['return20'] = df.return20
data = data.dropna()

In [365]:
print("数据集总长度：%d" %len(data))

数据集总长度：42736


In [366]:
# 划分训练集合测试集
train_size = int(len(data)*0.7)
train = data[:train_size]
test = data[train_size:]
print("训练集总长度：%d" %len(train))
print("训练集总长度：%d" %len(test))

训练集总长度：29915
训练集总长度：12821


In [367]:
# 取出训练集里return20在[-0.0008,0.0008]之间的样本
temp = train[(train['return20']<0.0008)&(train['return20']> -0.0008)]
print("范围内样本个数：%d" %len(temp))

范围内样本个数：26328


In [368]:
# 随机抽取部分该范围内的样本
temp = temp.sample(n = int(1*len(temp)))
print("降采样后样本个数：%d" %len(temp))

降采样后样本个数：26328


In [369]:
# 和范围外的样本合并
train = temp.append(train[(train['return20']>0.0008) | (train['return20']< -0.0008)])
print("降采样后训练集总长度：%d" %len(train))

降采样后训练集总长度：29915


In [370]:
features_train = train.drop(['return20'],axis=1)
features_train = ss.fit_transform(features_train)
labels_train = train['return20']

In [371]:
features_test = test.drop(['return20'],axis=1)
features_test = ss.fit_transform(features_test)
labels_test = test['return20']

In [372]:
# 训练Regression model
reg = RandomForestRegressor()
# reg = ExtraTreesRegressor()
# reg = GradientBoostingRegressor()
reg.fit(features_train, labels_train) 

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [373]:
predict_test = reg.predict(features_test)
predict_train = reg.predict(features_train)

In [386]:
# plt.scatter(predict_train, labels_train, s = 0.5)

In [387]:
# plt.scatter(predict_test, labels_test, s = 0.5)

In [376]:
# df_train = df.loc[train.index]
# df_train['predict'] = predict_train

In [384]:
# df_train[df_train['predict'] > 0.001]['bids20'].mean()

In [385]:
# df_train[df_train['predict'] < -0.001]['asks20'].mean()

In [379]:
df_test = df.loc[test.index]
df_test['predict'] = predict_test

In [397]:
df_test[df_test['predict'] > 0.001]['bids20'].mean()

-62.838571355684714

In [389]:
len1 = len(df_test[df_test['predict'] > 0.001])

In [395]:
df_test.sample(n = len1)['bids20'].mean()

-88.65075139137656

In [396]:
df_test[df_test['predict'] < -0.001]['asks20'].mean()

-2.7435045236570574

In [392]:
len2 = len(df_test[df_test['predict'] < -0.001])

In [393]:
df_test.sample(n = len2)['asks20'].mean()

-33.33298704065282